In [ ]:
# 

In [1]:
from pyspark.sql import SparkSession
from creds import dbUser, dbPasswd, dbHost, dbPort, om_admin_token

In [2]:
jar_path="/home/pengfei/git/OpenMetaIngestion/jars"
spark = (
    SparkSession.builder.master("local")
    .appName("localTestApp")
    .config(
        "spark.jars",
        f"{jar_path}/openmetadata-spark-agent-1.0.jar,{jar_path}/mysql-connector-java-8.0.30.jar")
    .config(
        "spark.extraListeners","org.openmetadata.spark.agent.OpenMetadataSparkListener")
    .config("spark.openmetadata.transport.hostPort", "http://datacatalog.casd.local")
    .config("spark.openmetadata.transport.type", "openmetadata")
    .config("spark.openmetadata.transport.jwtToken", om_admin_token)
    .config("spark.openmetadata.transport.pipelineServiceName", "test_pipeline_service1")
    .config("spark.openmetadata.transport.pipelineName", "my_pipeline_name")
    .config("spark.openmetadata.transport.pipelineSourceUrl", "http://casd.local/path/to/pipeline")
    .config("spark.openmetadata.transport.pipelineDescription", "My_ETL_Pipeline")
    .config("spark.openmetadata.transport.databaseServiceNames", "constance")
    .config("spark.openmetadata.transport.timeout", "30")
    .getOrCreate()
)


24/07/01 11:48:39 WARN Utils: Your hostname, pengfei-Virtual-Machine resolves to a loopback address: 127.0.1.1; using 10.50.2.80 instead (on interface eth0)
24/07/01 11:48:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/07/01 11:48:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# Read from MySQL Table
db_name="constance1"
table_name="ct_E2021_F2010_DCIR_BIO"
employee_df = (
    spark.read.format("jdbc")
    .option("url", f"jdbc:mysql://{dbHost}:{dbPort}/{db_name}")
    .option("driver", "com.mysql.cj.jdbc.Driver")
    .option("dbtable", table_name)
    .option("user", f"{dbUser}")
    .option("password", f"{dbPasswd}")
    .load()
)

employee_df.show(5)

24/07/01 11:48:51 WARN LogicalPlanSerializer: Unable to writeValueAsString
java.lang.reflect.InvocationTargetException
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at org.apache.commons.lang.reflect.MethodUtils.invokeMethod(MethodUtils.java:170)
	at org.apache.commons.lang.reflect.MethodUtils.invokeMethod(MethodUtils.java:131)
	at org.apache.commons.lang.reflect.MethodUtils.invokeMethod(MethodUtils.java:95)
	at io.openlineage.spark.agent.lifecycle.LogicalPlanSerializer.writeValueAsString(LogicalPlanSerializer.java:105)
	at io.openlineage.spark.agent.lifecycle.LogicalPlanSerializer.serialize(LogicalPlanSerializer.java:128)
	at io.openlineage.spark.agent.lifecycle.UnknownEntryFacet

+---+----+-----------+
|UID|Name|FLX_DIS_DTD|
+---+----+-----------+
+---+----+-----------+


In [4]:
employee_df.printSchema()

root
 |-- UID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- FLX_DIS_DTD: date (nullable = true)


In [5]:

# creat a new table with the dataframe employee_df
target_db_name = "pengfei_test"

employee_df.write.format("jdbc") \
.option("url", f"jdbc:mysql://{dbHost}:{dbPort}/{db_name}") \
.option("driver", "com.mysql.cj.jdbc.Driver") \
.option("dbtable", target_db_name) \
.option("user", f"{dbUser}") \
.option("password", f"{dbPasswd}") \
.mode("overwrite") \
.save()



24/07/01 11:51:01 WARN LogicalPlanSerializer: Unable to writeValueAsString) / 1]
java.lang.reflect.InvocationTargetException
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at org.apache.commons.lang.reflect.MethodUtils.invokeMethod(MethodUtils.java:170)
	at org.apache.commons.lang.reflect.MethodUtils.invokeMethod(MethodUtils.java:131)
	at org.apache.commons.lang.reflect.MethodUtils.invokeMethod(MethodUtils.java:95)
	at io.openlineage.spark.agent.lifecycle.LogicalPlanSerializer.writeValueAsString(LogicalPlanSerializer.java:105)
	at io.openlineage.spark.agent.lifecycle.LogicalPlanSerializer.serialize(LogicalPlanSerializer.java:128)
	at io.openlineage.spark.agent.lifecycle.UnknownEntr

In [ ]:
# Stop the Spark session
spark.stop()